In [2]:
from generate_psf import fit_gaussian, weighted_linear_fit
import numpy as np
import cupy as cp
import os
import glob
import skimage
import scipy
from skimage.filters.rank import threshold
from tqdm.auto import tqdm
import cv2
from i_o import Paths
import json
from daio.h5 import lazyh5
import importlib
import h5py
cmap = "gray"

In [2]:
# pn_psf = r"Y:/hpc/r/lfm2025/psfs"
pn_psfs = r"~/hpc-rw/lfm/psfs"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_psfs, '*'))), reverse=True) if os.path.exists(os.path.join(pn_psfs, rec, 'psf.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

20250701_1216_PSF_LB_noap_1
20250610_1636_PSF_LB_30k_wo_coverslip_1
20250530_1330_PSF_LB_redFB_1-30K_dim_isolated
20250509_1646_PSF_LB_redFB_1_30K_wo_coverslip_1
20250425_1320_PSF_LB_redFB_1-10000_2d_without_coverslip_3


In [3]:
# pn_rec = r"Y:/hpc/r/lfm2025/recordings"
pn_rec = r"~/hpc-r/lfm2025/recordings"
includes = ['']
excludes = []

recs = [os.path.basename(rec) for rec in sorted(glob.glob(os.path.expanduser(os.path.join(pn_rec, '*'))), reverse=True) if os.path.exists(os.path.join(pn_rec, rec, 'data.h5'))]
recs = [rec for rec in recs if all([s in rec for s in includes]) and not any([s in rec for s in excludes])]
print('\n'.join(recs))

20250701_1654_LB_bg_2fps
20250701_1651_LB_bg_40fps
20250701_1649_LB_bg_80fps
20250701_1638_LB_ZF_v552_f7_40fps_tail
20250701_1636_LB_ZF_v552_f7_80fps_tail
20250701_1633_LB_ZF_v552_f7_80fps_3
20250701_1630_LB_ZF_v552_f7_80fps_2
20250701_1628_LB_ZF_v552_f7_80fps_1
20250701_1625_LB_ZF_v552_f7_40fps_2
20250701_1622_LB_ZF_v552_f7_40fps_1
20250701_1613_LB_ZF_v552_f6_80fps_2
20250701_1610_LB_ZF_v552_f6_80fps_1
20250701_1607_LB_ZF_v552_f6_40fps_2
20250701_1605_LB_ZF_v552_f6_40fps_1
20250701_1602_LB_ZF_v552_f6_40fps_tail2
20250701_1600_LB_ZF_v552_f6_40fps_tail
20250701_1547_LB_ZF_v552_f5_80fps_2
20250701_1544_LB_ZF_v552_f5_80fps_1
20250701_1541_LB_ZF_v552_f5_40fps_2
20250701_1538_LB_ZF_v552_f5_40fps_1
20250701_1522_LB_ZF_v552_f4_80fps_1
20250701_1520_LB_ZF_v552_f4_40fps_2
20250701_1516_LB_ZF_v552_f4_40fps_1
20250701_1503_LB_ZF_v552_f3_80fps_1
20250701_1501_LB_ZF_v552_f3_40fps_2
20250701_1458_LB_ZF_v552_f3_40fps_1
20250701_1444_LB_ZF_v552_f2_80fps_1
20250701_1441_LB_ZF_v552_f2_40fps_3
20250701_1

In [3]:
pn_psfs = r"~/hpc-rw/lfm/psfs"
pn_rec = r"~/hpc-r/lfm2025/recordings"
pn_bg = r"~/hpc-rw/lfm/bg"
url_home = r"/home/lubo12/"
pn_out = r"~/hpc-rw/lfm/results"

dataset_name = "20250701_1501_LB_ZF_v552_f3_40fps_2"
psf_name = "20250701_1216_PSF_LB_noap_1"
bg_name = "20250701_1651_LB_bg_40fps.npy"

import i_o
importlib.reload(i_o)
from i_o import Paths

paths = Paths(dataset_name=dataset_name,
              psf_name = psf_name,
              bg_name=bg_name,
              pn_bg= pn_bg,
              pn_rec = pn_rec,
              pn_psfs=pn_psfs,
              pn_out=pn_out,
              url_home=url_home,
              )

In [5]:
from i_o import VolumeReader
reader = VolumeReader(paths.raw, "data")
reader.get_next()[1].shape

(2048, 2048)

In [8]:
paths.deconvolved[:-3]+"1.h5"

'/home/lubo12/hpc-rw/lfm/results/20250701_1501_LB_ZF_v552_f3_40fps_2/deconvolved1.h5'

In [21]:
with h5py.File(paths.deconvolved[:-3]+"1.h5", "w") as f:
    f["losses"] = old["losses"]
    f["n_iter"] = old["n_iter"]
    f["processed_indeces"] = old["processed_indices"]
    with h5py.File(paths.deconvolved, "r") as ff:
        shape = ff["data"].shape
        f.create_dataset("data", shape= (shape[0], shape[1], crop[1]-crop[0], crop[3]-crop[2]))
        for i in tqdm(range(shape[0])):
            f["data"][i, :, :,:] = ff["data"][i,:,crop[0]:crop[1],crop[2]:crop[3]]

  0%|          | 0/4020 [00:00<?, ?it/s]

In [9]:
import importlib
import i_o
importlib.reload(i_o)
from i_o import AsyncH5Writer
from daio.h5 import lazyh5

writer = AsyncH5Writer("test.h5")
test_dict = {"a": 1, "b": 2, 'c': {"d": 3, 'e': 4}}
writer.write_meta("test_dict", test_dict)

lazyh5("test.h5")

<IPython.core.display.JSON object>

In [11]:
import h5py
with h5py.File("test.h5", "r") as f:
    print(f["test_dict"]["c"].keys())

<KeysViewHDF5 ['d', 'e']>
